In [4]:
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import psycopg2
import seaborn as sns
import streamlit as st

In [5]:
DATABASE_URL = os.environ['DATABASE_URL']

In [6]:
connection = psycopg2.connect(DATABASE_URL, sslmode='require')
with connection:
    cursor = connection.cursor()

    # Get the number of questions
    query: str = "SELECT real, count(*) AS a FROM comments GROUP by real;"
    cursor.execute(query)
    comments_count: int = cursor.fetchall()

    # Get the general answers trend
    query: str = "SELECT answer, count(*) AS a FROM answers GROUP by answer;"
    cursor.execute(query)
    answers_count = cursor.fetchall()

    # Count number of different answers for each question
    query: str = """
    SELECT
        ca.id,
        ca.content,
        ca.real,
        ca.answer,
        count(*) AS count
    FROM (
        SELECT
            comments.id,
            comments.content,
            comments.real,
            answers.answer
        FROM
            comments
            INNER JOIN answers ON comments.id = answers.comment
        GROUP BY
            comments.id,
            comments.real,
            answers,
            answer) AS ca
    GROUP BY
        ca.id,
        ca.content,
        ca.real,
        ca.answer;
    """
    cursor.execute(query)
    answers = cursor.fetchall()
    cursor.close()

In [8]:
df = pd.DataFrame(answers, columns =['id', 'content', 'real', 'answer', 'count'])

In [32]:
df.head(10)

,id,content,real,answer,count
0,18245,prolly not. but only because idk you and i hav...,1,0,1
1,625,I hate you with the hate of a thousand hates. ;),1,0,1
2,19600,that is just awesome. theresplastic in spain a...,0,0,1
3,9843,And we are having a pig roast and beef and goo...,1,1,1
4,19280,You mean the fucking dog who ur talking with r...,0,0,1
5,17041,Of course! I sure would :),1,0,2
6,12452,Ooooh! Weasels! My favorite. At one time I h...,1,0,1
7,19409,If you could get any job what job would you h...,0,0,2
8,14754,Have you ever pretended your crush was with yo...,1,1,1
9,19259,fuck yeah bitch ;),0,1,1


In [31]:
df["count"].idxmax()

5

In [36]:
right_guesses: int = 0
wrong_guesses: int = 0
biggest_good_guesses: int = 0
biggest_wrong_guesses: int = 0
for i in df.index:
    if df['real'][i] == df['answer'][i]:
        right_guesses += df['count'][i]
        if df['count'][i] > biggest_good_guesses:
            biggest_good_guesses = df['count'][i]
            best_comment = (i, df["content"][i], df['count'][i])
    else:
        wrong_guesses += df['count'][i]
        if df['count'][i] > biggest_wrong_guesses:
            biggest_wrong_guesses = df['count'][i]
            worst_comment = (i, df["content"][i], df['count'][i])
print(best_comment)
print(worst_comment)

(7, 'If you could get any job  what job would you hate it with one eye closed? :)r  Leave a message so we', 2)
(5, 'Of course! I sure would :)', 2)
